# Setup

In [1]:
%reload_ext autoreload

In [2]:
%cd /home/kpmurphy/github/shifty
#from shifty.skax.logreg_flax import *
from shifty.skax.skax import *
#from shifty.tta.tta import *
from shifty.tta.gmm_data import *



/home/kpmurphy/github/shifty


2022-10-10 20:40:29.786225: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-10-10 20:40:30.169851: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-10 20:40:35.958474: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-10-10 20:40:35.958654: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib


<function logprior_fn at 0x7fa545132950>


In [3]:
# Silence WARNING:root:The use of `check_types` is deprecated and does not have any effect.
# https://github.com/tensorflow/probability/issues/1523
import logging

logger = logging.getLogger()


class CheckTypesFilter(logging.Filter):
    def filter(self, record):
        return "check_types" not in record.getMessage()


logger.addFilter(CheckTypesFilter())

In [4]:

import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=3)
import scipy.stats
import einops
from functools import partial

from functools import partial
from collections import namedtuple
import itertools
from itertools import repeat
from time import time
import chex
import typing
from typing import Any, Callable, Sequence

import jax
import jax.random as jr
import jax.numpy as jnp
from jax import vmap, grad, jit
from jax import lax, random, numpy as jnp
import jax.scipy as jsp

from flax.core import freeze, unfreeze
from flax import linen as nn
import flax

import jaxopt
import optax
import tensorflow as tf
import tensorflow_datasets as tfds

import torch
from torch.utils.data import TensorDataset, DataLoader

import distrax
from tensorflow_probability.substrates import jax as tfp
tfd = tfp.distributions
from tensorflow_probability.substrates.jax.distributions import MultivariateNormalFullCovariance as MVN

#jax.config.update("jax_enable_x64", False)


import sklearn.datasets
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LogisticRegression

RuntimeError: KeyboardInterrupt: 

In [7]:
import os 
cpu_count = os.cpu_count()
print(cpu_count)

# Run jax on multiple CPU cores
# https://github.com/google/jax/issues/5506
# https://stackoverflow.com/questions/72328521/jax-pmap-with-multi-core-cpu
import os 
#os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=90'

import jax
print(jax.devices())

96
[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]


# Code

In [60]:
from shifty.tta.label_space import *
from shifty.tta.data_generator import *
from shifty.tta.tta import *
from shifty.tta.estimators import *

In [80]:

key = jr.PRNGKey(0)
corr_source = 0.1
corr_targets = jnp.arange(0.1, 0.9, step=0.1)
ls = LabelSpace(nclasses=2, nfactors=2)
src_dist = GMMDataGenerator(key, corr_source, ls, nfeatures=10, noise_factor=1)

ntargets = ls.nclasses * ls.nfactors # network predicts joint output
nhidden = (10,) + (ntargets,) # set nhidden = () + (ntargets,) to get logistic regression
network = MLPNetwork(nhidden)
mlp = NeuralNetClassifier(network, key, ntargets, l2reg=1e-5, optimizer = "adam+warmup", 
        batch_size=32, num_epochs=20, print_every=0) 


In [93]:
#estimator = OracleEstimator(ls)
#estimator = UndaptedEstimator(mlp, ls)
estimator = EMEstimator(mlp, ls)

nsource_samples =500
Xs, Ys, As = src_dist.sample(key, nsource_samples)
estimator.fit_source(Xs, Ys, As, src_dist)




/home/kpmurphy/mambaforge/lib/python3.10/site-packages/distrax/_src/utils/conversion.py:143: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return x.astype(jnp.float_)


In [95]:
metrics = evaluate_estimator(key, src_dist, corr_targets, estimator)
print(metrics)

[0.1  0.1  0.12 0.2  0.15 0.17 0.21 0.13]


In [85]:
corr_targets

DeviceArray([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8], dtype=float32)